In [1]:
import pandas as pd
import numpy as np
import pickle as p
import time

In [2]:
data_all = pd.read_csv('../data_all.csv')

# Test with Subset of Data
np.random.seed(43)
item_subset = list(np.random.choice(data_all['Item_ID'].unique(), size=10, replace=False))
data_subset = data_all.loc[data_all['Item_ID'].isin(item_subset)].copy()

# # Test with All Data
# data_subset = data_all.copy()

test_project_id = 'test_1'

### `/train`

In [3]:
import requests
import json
from threading import Thread

# url = 'https://fillet.azurewebsites.net/train/'
url = 'http://localhost:5000/train/'

data_subset.to_parquet('data_subset.parquet')
files = {'data': open('data_subset.parquet', 'rb')}

payload = {'cv_acc': True, 'project_id': test_project_id, 'modeltype': 'default'}

Thread(target=requests.post,
       args=(url, ),
       kwargs={
           'data': payload,
           'files': files
       }).start()

time.sleep(10)
files['data'].close()

### `/query_progress`

In [11]:
import requests
import json

url = 'https://fillet.azurewebsites.net/query_progress/'
# url = 'http://localhost:5000/query_progress/'


payload = {'project_id': test_project_id}

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload,headers=headers,  timeout=3)

print('% Complete:',round(r.json()['pct_complete'],3))
print('CV % Complete:',round(r.json()['cv_progress'],3))
print('CV Done:',r.json()['cv_done'])

% Complete: 100.0
CV % Complete: 100.0
CV Done: 1


### `/batch_query_progress`

In [26]:
import requests
import json

url = 'https://fillet.azurewebsites.net/batch_query_progress/'
# url = 'http://localhost:5000/batch_query_progress/'


payload = {'project_id_ls': ['bbbb','aaaa','cccc','dddd','ffff','gggg']}

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload,headers=headers,  timeout=3)

print(r.json())

{'aaaa': {'cv_done': 1, 'cv_progress': 100.0, 'fi_done': 0, 'pct_complete': 100.0, 'project_items': [2531, 202, 1195, 1230, 3726, 16019, 3732, 3766, 2589, 3743], 'train_complete': [2531, 202, 1195, 1230, 3726, 16019, 3732, 3766, 2589, 3743]}, 'bbbb': {'cv_done': 1, 'cv_progress': 100.0, 'fi_done': 0, 'pct_complete': 100.0, 'project_items': [2531, 202, 1195, 1230, 3726, 16019, 3732, 3766, 2589, 3743], 'train_complete': [2531, 202, 1195, 1230, 3726, 16019, 3732, 3766, 2589, 3743]}, 'cccc': {'cv_done': 1, 'cv_progress': 100.0, 'fi_done': 0, 'pct_complete': 100.0, 'project_items': [2531, 202, 1195, 1230, 3726, 16019, 3732, 3766, 2589, 3743], 'train_complete': [2531, 202, 1195, 1230, 3726, 16019, 3732, 3766, 2589, 3743]}, 'dddd': 'project not found', 'ffff': 'project not found', 'gggg': 'project not found'}


### `/get_cv_results`

In [37]:
import requests
import json

url = 'https://fillet.azurewebsites.net/get_cv_results/'
# url = 'http://localhost:5000/get_cv_results/'


payload = {'project_id': test_project_id}

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload,headers=headers,  timeout=5)
pd.DataFrame().from_dict((json.loads(r.json())))

,item_id,avg_sales,r2_score,mae_score,mpe_score,rmse_score
0,2521.0,226.583542,0.766792,35.041464,0.154651,44.898795
1,1195.0,130.380447,0.715035,25.046061,0.192100,33.536204
2,7434.0,156.666092,0.818124,29.081072,0.185625,39.159035
3,7575.0,108.745979,0.906959,18.063956,0.166111,27.611357
4,350.0,191.013550,0.764161,26.895335,0.140803,38.506500
5,3766.0,92.051125,0.783402,16.271062,0.176761,21.938158
6,1230.0,9.089099,0.551014,3.840307,0.422518,5.283977
7,2867.0,948.296204,0.833086,116.963041,0.123340,147.183843
8,202.0,2130.495117,0.784608,320.904429,0.150624,426.974355
9,3732.0,127.392555,0.780559,19.902378,0.156229,26.698321


In [127]:
r.json()

{'status': 'incomplete'}

### `/optimize`

In [16]:
import json
with open('constraints_2.json') as f:
    constraints = json.load(f)
    
import requests
import json

url = 'http://localhost:5000/optimize/'
    
payload = {'project_id': 'test_1',
           'constraints': constraints, # two parts, first part constraint, second part price range limits
           'population': 300,
           'max_epoch': 20
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

In [15]:
best_prices = r.json()['report']
best_prices

[21071.37799638748, 0.0, 0.0]

### /get_opti_results

In [18]:

import requests
import json

# url = 'https://fillet.azurewebsites.net/get_cv_results/'
url = 'http://localhost:5000/get_opti_results/'


payload = {'project_id': test_project_id}

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload,headers=headers,  timeout=5)
r.json()






{'price_cols': ['Price_202',
  'Price_1195',
  'Price_1230',
  'Price_2531',
  'Price_2589',
  'Price_3726',
  'Price_3732',
  'Price_3743',
  'Price_3766',
  'Price_16019'],
 'report': [20198.128542137147, 0.0, 0.0],
 'report_info': ['estimated revenue of the optimized price',
  'number of hard constraints (including price ranges) violated',
  'number of soft constraints (preferences) violated'],
 'result': [2.15,
  3.55,
  3.7421662414971113,
  6.6,
  11.1,
  10.938662940022816,
  12.55,
  12.17143530454372,
  10.3,
  14.15]}

### `/predict/`

In [38]:
# Sample Actual Price 
from helper_functions import *

sales_data = data_subset

sales_data = optimize_memory(sales_data)

sales_data_wide = sales_data.set_index(
    ['Wk', 'Tier', 'Store', 'Item_ID']).unstack(level=-1).reset_index().copy()
sales_data_wide.columns = [
    ''.join(str(i) for i in col).strip()
    for col in sales_data_wide.columns.values
]
sales_data_wide = sales_data_wide.sort_values(
    ['Tier', 'Store', 'Wk'], ascending=True).reset_index(drop=True)

sales_data_wide_clean = sales_data_wide.dropna(axis=0).copy()
dataset = sales_data_wide_clean
price_columns = [
    col for col in sales_data_wide_clean.columns if col.startswith('Price')
]

a = dataset.sample(1)
a_dict = a[price_columns].iloc[0].to_dict()
a_input = {}
for key in a_dict.keys():
    a_input[key.split('_')[1]]=round(a_dict[key],2)

# print('Input Prices Quantities:')
# a_input

In [40]:
# Predict Sales Qty and Compare to Actual

import requests
import json

url = 'https://fillet.azurewebsites.net/predict/'
# url = 'http://localhost:5000/predict/'

payload = {'prices': a_input, 'project_id': test_project_id, 'modeltype': 'default'}

headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)
qty = r.json()['qty_estimates']
qty_outp = {}
for key in qty.keys():
    qty_outp[key] = round(qty[key], 0)
actual = a[[
    col for col in sales_data_wide_clean.columns if col.startswith('Qty_')
]].iloc[0].to_dict()
pd.DataFrame([qty_outp, actual], index=['Estimated', 'Actual']).transpose()

,Estimated,Actual
Qty_102,219.0,225.0
Qty_1195,99.0,98.0
Qty_1221,240.0,217.0
Qty_1230,6.0,6.0
Qty_16017,20.0,17.0
Qty_16018,111.0,143.0
Qty_16019,137.0,164.0
Qty_201,40.0,43.0
Qty_202,2801.0,2949.0
Qty_204,17.0,17.0


### `/get_feature_importances`

In [53]:
import requests
import json
import pandas as pd

# url = 'https://fillet.azurewebsites.net/get_feature_importances/'
url = 'http://localhost:5000/get_feature_importances/'

payload = {'project_id': test_project_id}

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload,headers=headers,  timeout=240)
pd.DataFrame(r.json()['202']).sort_values('importance', ascending=False)

,feature_name,importance
6,Price_3726,0.157354
3,Price_202,0.123959
4,Price_2531,0.117571
5,Price_2589,0.117251
1,Price_1230,0.104129
7,Price_3732,0.093999
9,Price_3766,0.093018
8,Price_3743,0.072451
0,Price_1195,0.060886
2,Price_16019,0.059381


### /detect_conflict


In [3]:
import requests
import json

# url = 'https://fillet.azurewebsites.net/detect_conflict/'
url = 'http://localhost:5000/detect_conflict/'
with open('constraints_2.json') as f:
    constraints = json.load(f)

payload = {'constraints': constraints
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

In [4]:
r.json()

{'conflict': 'No conflict'}